In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import sparse
from itertools import product, combinations
from pathlib import Path
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib

pd.options.display.max_rows = 500
#pd.options.display.max_columns = 4000

import sys
sys.path.append('../src/')


In [21]:
from extended_survey import process_people_df, process_places_df
from census import process_census
from constraints import get_ind_const, get_viv_const

In [4]:
survey_dir = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/')
personas_path = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/Personas19.CSV')
viviendas_path = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/Viviendas19.CSV')
census_iter_path = Path('../data/census_loc/ITER_19CSV20.csv')
census_resageburb_path = Path('../data/census_ageb_manz/RESAGEBURB_19CSV20.csv')

In [5]:
%%time
viviendas_cat = process_places_df(viviendas_path)
viviendas_full = pd.concat(viviendas_cat.values())
personas_cat = process_people_df(personas_path)
personas_full = pd.concat(personas_cat.values())
(
    df_mun, df_mun_min, df_mun_max,
    df_loc, df_loc_min, df_loc_max,
    df_agebs, df_agebs_min, df_agebs_max
) = process_census(census_iter_path, census_resageburb_path)

CPU times: user 20.8 s, sys: 1.83 s, total: 22.6 s
Wall time: 22.6 s


In [252]:
personas_full.RELIGION.value_counts()

Católica                                    263723
Protestante/cristiano evangélico             42657
Sin religión / Sin adscripción religiosa     29141
Religión no especificada                       877
Otros credos                                   268
Name: RELIGION, dtype: int64

In [246]:
personas_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336666 entries, 0 to 336665
Data columns (total 69 columns):
 #   Column                                                    Non-Null Count   Dtype   
---  ------                                                    --------------   -----   
 0   ID_PERSONA                                                336666 non-null  int64   
 1   ID_VIV                                                    336666 non-null  int64   
 2   FACTOR                                                    336666 non-null  int64   
 3   MUN                                                       336666 non-null  category
 4   CLAVIVP                                                   336666 non-null  category
 5   SEXO                                                      336666 non-null  category
 6   EDAD                                                      336666 non-null  category
 7   ENT_PAIS_NAC                                              336666 non-null  category

In [237]:
7/5500*100

0.1272727272727273

In [226]:
df_mun.POBTOT.min()

906

In [224]:
pd.crosstab(personas_full.MUN, personas_full.HLENGUA)

HLENGUA,Sí/Sí Español,Sí/No español,Sí/No especificado,No,No especificado,Blanco por pase
MUN,,,,,,
Abasolo,12,0,0,2760,2,163
Agualeguas,3,0,0,3206,0,112
Los Aldamas,0,0,0,1341,0,51
Allende,21,1,0,4684,0,236
Anáhuac,5,0,0,4182,5,243
Apodaca,178,0,3,17418,15,899
Aramberri,4,0,0,4393,0,190
Bustamante,6,0,0,3470,2,152
Cadereyta Jiménez,64,0,0,6064,3,291


In [205]:
personas_full.query('HLENGUA=="Sí/No español"')[['ID_VIV', 'MUN', 'SEXO', 'EDAD', 'CONACT', 'ASISTEN', 'RELIGION', 'SITUA_CONYUGAL', 'HLENGUA', 'DHSERSAL', 'AFRODES', 'TIE_TRASLADO_TRAB', 'MUN_TRAB']]

,ID_VIV,MUN,SEXO,EDAD,CONACT,ASISTEN,RELIGION,SITUA_CONYUGAL,HLENGUA,DHSERSAL,AFRODES,TIE_TRASLADO_TRAB,MUN_TRAB
7871,190040000069,Allende,F,25-49,Se dedica a los quehaceres del hogar,No,Protestante/cristiano evangélico,casada(o) sólo por el civil,Sí/No español,1000000000,No,Blanco por pase,Blanco por pase
64390,190100004034,El Carmen,F,25-49,Trabajó,No,Católica,unión libre,Sí/No español,0000000010,Sí,Más de 1 hora y hasta 2 horas,Guadalupe
72054,190120000902,Ciénega de Flores,M,65-130,No trabaja,No,Católica,viuda(o),Sí/No español,0000000010,No,Blanco por pase,Blanco por pase
102294,190180000987,García,M,25-49,Trabajó,No,Católica,separada(o),Sí/No español,1000000000,No,31 minutos a 1 hora,García
106898,190180002281,García,F,65-130,Se dedica a los quehaceres del hogar,No,Católica,viuda(o),Sí/No español,0000000010,No,Blanco por pase,Blanco por pase
172717,190260003242,Guadalupe,M,25-49,Trabajó,No,Sin religión / Sin adscripción religiosa,unión libre,Sí/No español,0000000010,No,Más de 1 hora y hasta 2 horas,Guadalupe
180890,190280000264,Higueras,M,25-49,Trabajó,No,Católica,casada(o) sólo religiosamente,Sí/No español,1000000000,No,Más de 1 hora y hasta 2 horas,Apodaca
283990,190450001597,Salinas Victoria,M,65-130,Es pensionada(o) o jubilada(o),No,Católica,soltera(o),Sí/No español,0000000010,No,Blanco por pase,Blanco por pase
310795,190480001061,Santa Catarina,F,25-49,No trabaja,No,Católica,casada(o) civil y religiosamente,Sí/No español,1000000000,No,Blanco por pase,Blanco por pase
332341,190490001295,Santiago,F,25-49,Se dedica a los quehaceres del hogar,No,Protestante/cristiano evangélico,casada(o) sólo por el civil,Sí/No español,1000000000,No,Blanco por pase,Blanco por pase


In [221]:
viviendas_full.loc[190490001295].NUMPERS

5

In [180]:
personas_full.query('EDAD=="Unknown"')[['ID_VIV', 'MUN', 'SEXO', 'EDAD', 'CONACT', 'ASISTEN', 'RELIGION', 'SITUA_CONYUGAL', 'HLENGUA', 'DHSERSAL', 'AFRODES', 'TIE_TRASLADO_TRAB']]

,ID_VIV,MUN,SEXO,EDAD,CONACT,ASISTEN,RELIGION,SITUA_CONYUGAL,HLENGUA,DHSERSAL,AFRODES,TIE_TRASLADO_TRAB
22703,190060001564,Apodaca,F,Unknown,No especificado,No,Católica,No especificado,No,0000000010,No,Blanco por pase
32959,190060004423,Apodaca,F,Unknown,Trabajó,No,Católica,casada(o) sólo por el civil,No,1000000000,No,16 a 30 minutos
32960,190060004423,Apodaca,F,Unknown,Trabajó,Sí,Católica,soltera(o),No,0000000010,No,16 a 30 minutos
32961,190060004423,Apodaca,M,Unknown,Declara que busca trabajo / se rescata que tr...,Sí,Católica,soltera(o),Sí/No especificado,0000000010,No,16 a 30 minutos
54110,190100001108,El Carmen,F,Unknown,No especificado,No especificado,Religión no especificada,No especificado,No especificado,0000000001,No especificado,Blanco por pase
54112,190100001108,El Carmen,M,Unknown,No especificado,No especificado,Religión no especificada,No especificado,No especificado,0000000001,No especificado,Blanco por pase
57495,190100002033,El Carmen,M,Unknown,No especificado,No,Religión no especificada,No especificado,No,0000000001,No especificado,Blanco por pase
60940,190100003093,El Carmen,F,Unknown,Es estudiante,Sí,Sin religión / Sin adscripción religiosa,No especificado,No,1000000000,No,Blanco por pase
61449,190100003238,El Carmen,M,Unknown,No especificado,No especificado,Religión no especificada,No especificado,No especificado,1000000000,No,Blanco por pase
115878,190180004886,García,F,Unknown,No especificado,No especificado,Católica,No especificado,No especificado,1000000000,No,Blanco por pase


In [22]:
viv_const = get_viv_const()

In [149]:
viviendas_full.FINANCIAMIENTO1.value_counts()

Usó sus propios recursos.                         32767
Blanco por pase                                   30894
INFONAVIT                                         27704
Banco                                              4045
Otra institución                                   1400
FOVISSSTE                                           795
Le prestó un familiar, amiga(o) o prestamista.      521
PEMEX                                               116
FONHAPO                                              50
No especificado                                      23
Name: FINANCIAMIENTO1, dtype: int64

In [144]:
pd.crosstab(viviendas_full.NUMPERS, viviendas_full.INGTRHOG)

INGTRHOG,0-999,"1,000-4,999","5,000-9,999","10,000-19,999","20,000-39,999","40,000-79,999","80,000-149,999","150,000yMas",No especificado,Blanco por pase
NUMPERS,,,,,,,,,,
1,493,1437,3032,1440,543,143,34,5,100,5280
2,1086,2127,4643,3988,1851,574,116,37,213,5801
3,914,1660,5816,5781,2565,673,164,52,280,1947
4,795,1478,5951,7028,3599,993,205,89,301,1161
5,517,913,3605,4804,2582,667,150,52,166,631
6,202,355,1286,1979,1216,230,61,25,76,252
7,96,132,414,775,545,131,20,12,23,95
8,40,54,147,331,313,69,16,5,7,34
9,11,16,63,143,158,31,2,3,12,12


In [139]:
viviendas_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98315 entries, 190010000001 to 190510000884
Data columns (total 74 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   FACTOR            98315 non-null  int64   
 1   MUN               98315 non-null  category
 2   CLAVIVP           98315 non-null  category
 3   PAREDES           98315 non-null  category
 4   TECHOS            98315 non-null  category
 5   PISOS             98315 non-null  category
 6   COCINA            98315 non-null  category
 7   CUADORM           98315 non-null  category
 8   TOTCUART          98315 non-null  category
 9   LUG_COC           98315 non-null  category
 10  COMBUSTIBLE       98315 non-null  category
 11  ESTUFA            98315 non-null  category
 12  ELECTRICIDAD      98315 non-null  category
 13  FOCOS             98315 non-null  category
 14  FOCOS_AHORRA      98315 non-null  category
 15  AGUA_ENTUBADA     98315 non-null  category
 16  ABA_

In [20]:
viviendas_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98315 entries, 190010000001 to 190510000884
Data columns (total 74 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   FACTOR            98315 non-null  int64   
 1   MUN               98315 non-null  category
 2   CLAVIVP           98315 non-null  category
 3   PAREDES           98315 non-null  category
 4   TECHOS            98315 non-null  category
 5   PISOS             98315 non-null  category
 6   COCINA            98315 non-null  category
 7   CUADORM           98315 non-null  category
 8   TOTCUART          98315 non-null  category
 9   LUG_COC           98315 non-null  category
 10  COMBUSTIBLE       98315 non-null  category
 11  ESTUFA            98315 non-null  category
 12  ELECTRICIDAD      98315 non-null  category
 13  FOCOS             98315 non-null  category
 14  FOCOS_AHORRA      98315 non-null  category
 15  AGUA_ENTUBADA     98315 non-null  category
 16  ABA_

In [18]:
viviendas_full.TIPOHOG.value_counts()

Hogar Nuclear (Familiar)            64353
Hogar Ampliado (Familiar)           20105
Hogar unipersonal (No familiar)     12507
Hogar corresidente (No familiar)      669
Hogar Compuesto (Familiar)            620
Hogar no especificado (Familiar)       31
No se sabe la composición              30
Name: TIPOHOG, dtype: int64

In [17]:
personas_full.columns

Index(['ID_PERSONA', 'ID_VIV', 'FACTOR', 'MUN', 'CLAVIVP', 'SEXO', 'EDAD',
       'ENT_PAIS_NAC', 'PARENTESCO', 'NACIONALIDAD', 'SERSALUD', 'AFRODES',
       'DHSERSAL', 'RELIGION', 'DIS', 'HLENGUA', 'ELENGUA', 'PERTE_INDIGENA',
       'ASISTEN', 'ENT_PAIS_ASI', 'MUN_ASI', 'TIE_TRASLADO_ESCU',
       'MED_TRASLADO_ESC_Automóvil o camioneta', 'MED_TRASLADO_ESC_Bicicleta',
       'MED_TRASLADO_ESC_Blanco por pase', 'MED_TRASLADO_ESC_Caminando',
       'MED_TRASLADO_ESC_Camión, autobús, combi, colectivo',
       'MED_TRASLADO_ESC_Metro, tren ligero, tren suburbano',
       'MED_TRASLADO_ESC_Metrobús (autobús en carril confinado)',
       'MED_TRASLADO_ESC_Motocicleta o motoneta',
       'MED_TRASLADO_ESC_No especificado', 'MED_TRASLADO_ESC_Otro',
       'MED_TRASLADO_ESC_Taxi (App Internet)',
       'MED_TRASLADO_ESC_Taxi (sitio, calle, otro)',
       'MED_TRASLADO_ESC_Transporte escolar', 'NIVACAD', 'ALFABET',
       'ENT_PAIS_RES_5A', 'MUN_RES_5A', 'SITUA_CONYUGAL', 'CONACT',
       'OC

In [6]:
# Map people to households
viv_per_dict = {}
for id_viv, group in personas_full.groupby('ID_VIV'):
    viv_per_dict[id_viv] = group[['ID_PERSONA', 'SEXO', 'EDAD', 'PARENTESCO']]

household_set = set()
for viv_id, viv_df in viv_per_dict.items():
    # print(tuple(viv_df.PARENTESCO.value_counts()))
    household_set.update([tuple(viv_df.PARENTESCO.value_counts())])

In [11]:
viv_per_dict[190010000001]

,ID_PERSONA,SEXO,EDAD,PARENTESCO
0,19001000000100001,F,8-11,Hija(o)
1,19001000000100002,M,25-49,Esposa(o)
2,19001000000100003,M,0-2,Hija(o)
3,19001000000100004,F,6-7,Hija(o)
4,19001000000100005,F,25-49,Jefa(e)


In [10]:
household_set

{(1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0),
 (1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0),
 (1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0),
 (1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0),
 (1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 